# 模型构造

让我们回顾一下在[“多层感知机的简洁实现”](../chapter_deep-learning-basics/mlp-gluon.ipynb)一节中含单隐藏层的多层感知机的实现方法。我们首先构造`Sequential`实例，然后依次添加两个全连接层。其中第一层的输出大小为256，即隐藏层单元个数是256；第二层的输出大小为10，即输出层单元个数是10。我们在上一章的其他
节中也使用了`Sequential`类构造模型。这里我们介绍另外一种基于`Module`类的模型构造方法：它让模型构造更加灵活。


## 继承`Module`类来构造模型

`Module`类是`nn`模块里提供的一个模型构造类，我们可以继承它来定义我们想要的模型。下面继承`Module`类构造本节开头提到的多层感知机。这里定义的`MLP`类重载了`Module`类的`__init__`函数和`forward`函数。它们分别用于创建模型参数和定义前向计算。前向计算也即正向传播。

In [25]:
import torch
from torch import nn

class MLP(nn.Module):
    # 声明带有模型参数的层，这里声明了两个全连接层
    def __init__(self, **kwargs):
        # 调用MLP父类Module的构造函数来进行必要的初始化。这样在构造实例时还可以指定其他函数
        # 参数，如“模型参数的访问、初始化和共享“一节将介绍的模型参数params
        super(MLP, self).__init__(**kwargs)
        self.hidden = nn.Linear(20, 256) # 隐藏层
        self.activation = nn.ReLU()
        self.output = nn.Linear(256, 10) # 输出层
        
    # 定义模型的前向计算，即如何根据输入x计算返回所需要的模型输出
    def forward(self, x):
        x = self.hidden(x)
        x = self.activation(x)
        x = self.output(x)
        return x

以上的`MLP`类中无须定义反向传播函数。系统将通过自动求梯度而自动生成反向传播所需的`backward`函数。

我们可以实例化`MLP`类得到模型变量`net`。下面的代码初始化`net`并传入输入数据`X`做一次前向计算。其中，`net(X)`会调用`MLP`继承自`Module`类的`__call__`函数，这个函数将调用`MLP`类定义的`forward`函数来完成前向计算。

In [31]:
X = torch.rand(2, 20)
net = MLP()
net(X)

tensor([[ 0.2840,  0.1051,  0.0097, -0.1576, -0.1534,  0.0705,  0.2980, -0.1177,
         -0.0814, -0.1636],
        [ 0.3106, -0.0251, -0.0076, -0.0331, -0.0405, -0.0605,  0.2905, -0.1054,
          0.0534, -0.2083]], grad_fn=<AddmmBackward>)

注意，这里并没有将`Module`类命名为`Layer`（层）或者`Model`（模型）之类的名字，这是因为该类是一个可供自由组建的部件。它的子类既可以是一个层（如nn提供的`Linear`类），又可以是一个模型（如这里定义的`MLP`类），或者是模型的一个部分。我们下面通过两个例子来展示它的灵活性。

## `Sequential`类继承自`Module`类

我们刚刚提到，`Module`类是一个通用的部件。事实上，`Sequential`类继承自`Module`类。当模型的前向计算为简单串联各个层的计算时，可以通过更加简单的方式定义模型。这正是`Sequential`类的目的：它提供`add_module`函数来逐一添加串联的`Module`子类实例，而模型的前向计算就是将这些实例按添加的顺序逐一计算。

下面我们实现一个与`Sequential`类有相同功能的`MySequential`类。这或许可以帮助读者更加清晰地理解`Sequential`类的工作机制。

In [24]:
class MySequential(nn.Module):
    def __init__(self, ** kwargs):
        super(MySequential, self).__init__(**kwargs)
        
    def add_module(self, name, module):
        # module是一个Module子类实例，将设它有独一无二的名字。我们将它保存在Module类的
        # 成员变量__modules里，其类型是OrderedDict。
        self._modules[name] = module
    
    def forward(self, x):
        # OrderDict保证会按照成员添加时的顺序遍历成员
        for module in self._modules.values():
            x = module(x)
        return x

我们用`MySequential`类来实现前面描述的`MLP`类，并使用随机初始化的模型做一次前向计算。

In [33]:
net = MySequential()
net.add_module("hidden", nn.Linear(20, 256))
net.add_module("activation", nn.ReLU())
net.add_module("output", nn.Linear(256, 10))
net(X)

tensor([[-0.0298,  0.0742,  0.2151,  0.2104, -0.2136,  0.1869,  0.0108,  0.0742,
          0.0199, -0.0544],
        [-0.0400,  0.0309,  0.0342,  0.1350, -0.1992,  0.1565, -0.1750, -0.0003,
          0.0788, -0.0534]], grad_fn=<AddmmBackward>)

*注：MySequential的结果与MLP的结果不同是因为Linear层的参数是随机初始化的。*

可以观察到这里`MySequential`类的使用跟[“多层感知机的简洁实现”](../chapter_deep-learning-basics/mlp-gluon.ipynb)一节中`Sequential`类的使用没什么区别。


## 构造复杂的模型

虽然`Sequential`类可以使模型构造更加简单，且不需要定义`forward`函数，但直接继承`Module`类可以极大地拓展模型构造的灵活性。下面我们构造一个稍微复杂点的网络`FancyMLP`。在这个网络中，我们通过`get_constant`函数创建训练中不被迭代的参数，即常数参数。在前向计算中，除了使用创建的常数参数外，我们还使用`torch`的函数和Python的控制流，并多次调用相同的层。

In [39]:
class FancyMLP(nn.Module):
    def __init__(self, **kwargs):
        super(FancyMLP, self).__init__(**kwargs)
        self.rand_weight = nn.Parameter(torch.rand(20, 20), requires_grad=False)
        self.linear = nn.Linear(20, 20)
        self.activation = nn.ReLU()
    
    def forward(self, x):
        x = self.linear(x)
        x = self.activation(x)
        # 使用创建的常数参数，以及nn的relu函数和mm函数
        x = torch.relu(x.mm(self.rand_weight.data) + 1)
        # 复用全连接层。等价于两个全连接层共享参数
        x = self.linear(x)
        x = self.activation(x)
        # 控制流，这里我们需要调用item函数来返回标量进行比较
        while x.norm().item() > 1:
            x /= 2
        if x.norm().item() < 0.8:
            x *= 10
        return x.sum()

在这个`FancyMLP`模型中，我们使用了常数权重`rand_weight`（注意它不是模型参数）、做了矩阵乘法操作（`torch.dot`）并重复使用了相同的`Linear`层。下面我们来测试该模型的随机初始化和前向计算。

In [40]:
net = FancyMLP()
net(X)

tensor(3.7902, grad_fn=<SumBackward0>)

因为`FancyMLP`和`Sequential`类都是`Module`类的子类，所以我们可以嵌套调用它们。

In [44]:
class NestMLP(nn.Module):
    def __init__(self, **kwargs):
        super(NestMLP, self).__init__(**kwargs)
        self.net = nn.Sequential(
            nn.Linear(20, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU()
        )
        
        self.linear3 = nn.Linear(32, 16)
        self.activation3 = nn.ReLU()
    
    def forward(self, x):
        return self.activation3(self.linear3(self.net(x)))

net = nn.Sequential(NestMLP(), nn.Linear(16, 20), FancyMLP())

net(X)

tensor(17.1225, grad_fn=<SumBackward0>)

## 小结

* 可以通过继承`Module`类来构造模型。
* `Sequential`类继承自`Module`类。
* 虽然`Sequential`类可以使模型构造更加简单，但直接继承`Module`类可以极大地拓展模型构造的灵活性。


## 练习

* 如果不在`MLP`类的`__init__`函数里调用父类的`__init__`函数，会出现什么样的错误信息？
* 如果去掉`FancyMLP`类里面的`item`函数，会有什么问题？
* 如果将`NestMLP`类中通过`Sequential`实例定义的`self.net`改为`self.net = [nn.Linear(20, 64), nn.ReLU(), nn.Linear(64, 32),nn.ReLU()]`，会有什么问题？


## 扫码直达[讨论区](https://discuss.gluon.ai/t/topic/986)

![](../img/qr_model-construction.svg)